#### Фишки по работе с кодом в формате строки

Здесь рассматриваются инструменты по различным представлениям кода. Так, например, в исходной статье модели [UniXCoder](https://arxiv.org/pdf/2203.03850), специализированной на нескольких задачах по обработке кодовых последовательностей, описывается подход подачи комментария к коду и представления кода в формате AST (+ prefix токен в начале, характерирзующий Encoder/Decoder в зависимости от типа downstream задачи). 

*INPUT* = Prefix (task token) + Comment + Flatten AST

В статьте отмечается преимущество подобного подхода в сравнении с базовыми подходами в codeBERT (с точки зрения метрик).

Поэтому есть смысл подавать в сеть различные представления кода, чтобы модель смогла лучше выучивать не только семантические, но и структурные закономерности токенов, что важно при понимании текста кода. 

Поэтому рассмотрение подобных инструментов может быть полезным с точки зрения подготовки данных к обучению.

#### AST-tree

*Абстрактное синтаксическое дерево (AST, Abstract Syntax Tree)* — это структура данных, которая представляет исходный код программы в виде дерева. Узлы дерева соответствуют конструкциям языка, таким как выражения, операторы, объявления переменных и т. д.

Пример для Python:

In [1]:
import ast

# Исходный код
source_code = """
x = 10
y = 20
z = x + y
"""

# Парсим код в AST
tree = ast.parse(source_code)

# Выводим дерево
print(ast.dump(tree, indent=4))


Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=Constant(value=10)),
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=Constant(value=20)),
        Assign(
            targets=[
                Name(id='z', ctx=Store())],
            value=BinOp(
                left=Name(id='x', ctx=Load()),
                op=Add(),
                right=Name(id='y', ctx=Load())))],
    type_ignores=[])


Для C++ (сгенерировано с помощью chatGPT):

```bash
clang -Xclang -ast-dump -fsyntax-only code.cpp
```
```cpp
int main() {
    int x = 10;
    int y = x + 5;
    return y;
}
```
```css
TranslationUnitDecl
|-FunctionDecl main
    |-CompoundStmt
        |-DeclStmt
            |-VarDecl x
        |-DeclStmt
            |-VarDecl y
        |-ReturnStmt
```

Для JavaScript и других языков аналогично есть подобные инструменты, позволяющие проводить анализ AST дерева для кода.

Рассмотрим элементы python-библиотек для анализа подобного кода:

*Java*, например

In [10]:
import javalang

# Пример Java-кода
code = """
class Example {
    public static void main(String[] args) {
        int x = 10;
    }
}
"""
tree = javalang.parse.parse(code)
print(tree)


CompilationUnit(imports=[], package=None, types=[ClassDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=[LocalVariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=10), name=x)], modifiers=set(), type=BasicType(dimensions=[], name=int))], documentation=None, modifiers={'public', 'static'}, name=main, parameters=[FormalParameter(annotations=[], modifiers=set(), name=args, type=ReferenceType(arguments=None, dimensions=[None], name=String, sub_type=None), varargs=False)], return_type=None, throws=None, type_parameters=None)], documentation=None, extends=None, implements=None, modifiers=set(), name=Example, type_parameters=None)])


Встроенные python-методы есть не для всех ЯП, однако для имеющихся датасетов на Python и Java продемонстрированные библиотеки могут быть полезны для начального датасета и решения задачи.

При наличии дополнительных датасетов (например, при наличии кода на C++) можно использовать встроенные методы рассматриваемых языков.

#### Универсальные парсер Tree-sitter

*Tree-sitter* — это современный парсер для анализа синтаксических деревьев исходного кода. Он поддерживает множество языков программирования и позволяет работать с кодом на уровне структурированных синтаксических элементов. Tree-sitter быстро генерирует абстрактное синтаксическое дерево (AST) и предоставляет API для работы с ним.

Для Python:

In [ ]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser

PY_LANGUAGE = Language(tspython.language())
parser = Parser(PY_LANGUAGE)

tree = parser.parse(bytes("""def foo():	if bar:	baz()""", "utf8"))

def traverse_tree(node, depth=0):
    print("  " * depth + f"{node.type} [{node.start_point} - {node.end_point}]")
    for child in node.children:
        traverse_tree(child, depth + 1)
        
traverse_tree(tree.root_node)

module [Point(row=0, column=0) - Point(row=0, column=24)]
  function_definition [Point(row=0, column=0) - Point(row=0, column=24)]
    def [Point(row=0, column=0) - Point(row=0, column=3)]
    identifier [Point(row=0, column=4) - Point(row=0, column=7)]
    parameters [Point(row=0, column=7) - Point(row=0, column=9)]
      ( [Point(row=0, column=7) - Point(row=0, column=8)]
      ) [Point(row=0, column=8) - Point(row=0, column=9)]
    : [Point(row=0, column=9) - Point(row=0, column=10)]
    ERROR [Point(row=0, column=11) - Point(row=0, column=13)]
      identifier [Point(row=0, column=11) - Point(row=0, column=13)]
    block [Point(row=0, column=14) - Point(row=0, column=24)]
      expression_statement [Point(row=0, column=14) - Point(row=0, column=24)]
        assignment [Point(row=0, column=14) - Point(row=0, column=24)]
          identifier [Point(row=0, column=14) - Point(row=0, column=17)]
          : [Point(row=0, column=17) - Point(row=0, column=18)]
          type [Point(row=0,

Для C++ и других языков смотреть непостредственно внутри соответствующих им модулей

#### FlowСhart для отображения структуры кода

In [4]:
from pyflowchart import Flowchart

# Исходный код
code = """
def example(x):
    if x > 0:
        print("Positive")
    else:
        print("Non-positive")
    print("Done")
"""

# Генерация блок-схемы
fc = Flowchart.from_code(code)
print(fc.flowchart())  # Вывод блок-схемы в формате flowchart.js


st3=>start: start example
io5=>inputoutput: input: x
cond9=>condition: if (x > 0)
sub13=>subroutine: print('Positive')
sub20=>subroutine: print('Done')
e22=>end: end example
sub17=>subroutine: print('Non-positive')

st3->io5
io5->cond9
cond9(yes)->sub13
sub13->sub20
sub20->e22
cond9(no)->sub17
sub17->sub20



Генерирует данные в формате .js, в целом интересная задумка, но есть проблемы с поддержкой на других языках программирования

#### Резюме

* Существуют методы разнообразного форматирования кода перед подачей в языковую модель, в том числе в виде python-библиотек и с поддержкой разных языков программирования

* Требует более тщательного анализа и подбора архитектур в связи с разными форматами данных